In [3]:
import pandas as pd
history_df = pd.read_csv('CAC 40 - history.csv', index_col='Date')

In [5]:
import numpy as np
import pandas as pd

def momentum(history, period, differential='last', method='normal'):
    """
    Calculates the momentum of a time series data over a specified period.

    Momentum measures the rate of change in the price of a security over a specific period.
    It is commonly used by traders and analysts to identify trends and potential buying or selling signals.

    Parameters:
    - history (pd.Series or pd.DataFrame): Time series data representing the price or value of a security.
    - period (int): The number of data points over which momentum is calculated.
    - differential (str, default='last'): Determines how the reference value is calculated within the period.
        - 'last': Uses the last value within the rolling window as the reference value.
        - 'mean': Uses the mean (average) value within the rolling window as the reference value.
    - method (str, default='normal'): Determines the method of calculating momentum.
        - 'normal': Calculates the difference between the current value and the reference value.
        - 'roc': Calculates the Rate of Change (ROC) as a percentage.
        - 'roclog': Calculates the logarithmic Rate of Change (ROC) as a percentage.

    Returns:
    - pd.Series: Series containing the calculated momentum values.
    """
    # Calculate the reference values based on the selected method
    if differential == 'last':
        ctx = history.rolling(window=period).apply(lambda x: x[0], raw=True).dropna()
    elif differential == 'mean':
        ctx = history.rolling(window=period).mean().dropna()

    # Reindex the original history to align with the reference values
    ct = history.reindex(ctx.index)

    # Calculate momentum based on the selected method
    if method == 'normal':
        mo = ct - ctx
    elif method == 'roc':
        mo = 100 * ct / ctx
    elif method == 'roclog':
        mo = 100 * np.log(np.array(ct / ctx))
        mo = pd.Series(mo, index=history.index[-len(ct):])
    return mo


In [10]:
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import matplotlib.ticker
import pandas as pd

def plot(history, period, differential, method, n_stock): 
    """
    Plots the first and last momentum for the given historical data.
    
    Args:
        history (pd.DataFrame): Historical data containing stock information.
        period (int): The period for momentum calculation.
        differential (str): The method for differential calculation, can be 'last' or 'mean'.
        method (str): The method for momentum calculation, can be 'normal', 'roc', or 'roclog'.
        n_stock (int): The number of top and flop stocks to display.
    """
    # Create a series to store momentum results for each stock
    df = pd.Series()
    # Calculate momentum for each stock in the history
    for ticker in history.columns.unique():
        df[ticker] = momentum(history[ticker], period=period, differential=differential, method=method).iloc[-1]
    # Drop NaN values
    df = df.dropna()

    # Select the last n_stock values for the first and second plot (strongest and weakestmomentum)
    df_first = df.sort_values().tail(n_stock)
    df_last = df.sort_values().head(n_stock)

    # Create the figure and two subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
    df_first.sort_values(ascending=True).plot(kind='barh', color='g', ax=ax2)
    df_last.sort_values(ascending=False).plot(kind='barh', color='r', ax=ax1)

    # Display y-axis on the right side of the second graph
    ax2.yaxis.set_ticks_position('right')

    # Set x-axis limits for each graph
    ax1.set_xlim([df_last.min()*1.3, 0])
    ax2.set_xlim([0, df_first.max()*1.3])

    # Display x-axis scale only as integers
    ax1.xaxis.set_major_locator(matplotlib.ticker.MaxNLocator(integer=True))
    ax2.xaxis.set_major_locator(matplotlib.ticker.MaxNLocator(integer=True))

    # Display variations directly on the graph
    for i, v in enumerate(df_last.sort_values(ascending=False)):
        ax1.text(df_last.min()*0.18+v, i, f"{v:.2f}", color='r')
    for i, v in enumerate(df_first):
        ax2.text(df_first.min()*0.05+v, i, f"{v:.2f}", color='g')

    # Add a title to the figure
    fig.suptitle('First and last momentum', y=0.95)

    # Adjust the gap between the two graphs
    fig.subplots_adjust(wspace=0.05)
    
    plt.show()

def show(history):
    """Display interactive controls for selecting frequency and time range"""
    # Creating interactive controls for selecting frequency and time range
    controls = widgets.interactive(
        plot,
        history=widgets.fixed(history),
        n_stock=widgets.IntText(value=5, description='Stocks:', disabled=False),
        period=widgets.IntText(value=20, description='Periods:', disabled=False),
        differential=widgets.Select(options=['last', 'mean'], value='last', description='Differential:'),
        method=widgets.Select(options=['normal', 'roc', 'roclog'], value='normal', description='Method:'),
    )

    # Displaying the interactive controls
    display(controls)

In [11]:
show(history=history_df)

interactive(children=(IntText(value=20, description='Periods:'), Select(description='Differential:', options=(…